## PRACTICA GUIADA

### PARTE I: Limpieza y transformación de datos

Esta práctica se propone brindar un catálogo de métodos y funciones en Pandas y Pyhton que podrán ser útiles a la hora de encarar tareas de limpieza de datos. 

En general, podemos identificar seis tipos de tareas u operaciones que aplicamos a los datos en la etapa de limpieza.

1. Estandarización de categorías (homogeneización)
2. Resolución de problemas de formato
3. Asignación de formatos adecuados (dtype)
4. Corrección de valores erróneos
5. Completar datos faltantes (missing data imputation)
6. Organización correcta del dataset (tidy data)

Las funciones y métodos presentados abarcan una o varias de estas operaciones.

### Remover duplicados

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

* `duplicated()` devuelve un booleano identificando los casos duplicados.
* `drop_duplicates()` devuelve el `DataFrame` sin los casos duplicados

In [ ]:
data.duplicated()

In [ ]:
# Podemos definir algunos parámetros:

data.duplicated(['k1'],keep='last')

In [ ]:
data.drop_duplicates()

In [ ]:
data[~data.duplicated()] == data.drop_duplicates()

* Se puede utilizar `drop_duplicates()` para eliminar duplicados en una sola columna o en un set de columnas.

In [ ]:
data['k3'] = range(7)
data

In [ ]:
data.drop_duplicates(['k1'])

In [ ]:
data.drop_duplicates(['k1', 'k2'])

### Mapear y transformar los datos
A partir de un diccionario, se puede crear una nueva columna para un Dataframe donde las claves del mismo se vinculen con una de las series y los valores formen parte de la nueva columna.

In [ ]:
data = pd.DataFrame({'platos': ['panceta', 'bondiola', 'panceta', 'Pastrami',
                           'pavita', 'Panceta', 'pastrami', 'jamon crudo',
                           'nova lox'],
                  'peso': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
data.platos.unique()

* La idea es ahora poder asignar a cada `plato` un determinado `animal`. Una opción es hacerlo con los métodos `.map()` o `.apply()`.

Repaso:

    1)  pd.DataFrame.apply: Opera sobre filas o columnas completas
    2)  pd.DataFrame.applymap: Opera sobre cada uno de los elementos del Dataframe
    3)  pd.Series.apply: Opera sobre cada uno de los elementos de la Serie. 
    4)  pd.Series.map: Opera sobre cada uno de los elementos de la Serie, muy similar a Series.apply. Una diferencia importante es que map puede recibir una serie o un diccionario, además de una función.

In [ ]:
plato_a_animal = {
  'panceta': 'cerdo',
  'bondiola': 'cerdo',
  'pastrami': 'vaca',
  'pavita': 'pavo',
  'jamon crudo': 'cerdo',
  'nova lox': 'salmon'
}

In [ ]:
data['platos'] = data['platos'].map(str.lower)
data['animal'] = data['platos'].map(plato_a_animal)
data

* Podríamos también pasar una función que haga todo en un solo paso:

In [ ]:
data['platos'].map(lambda x: plato_a_animal[x.lower()])

* En este caso, funciona también con Series.apply():

In [ ]:
data['platos'].apply(lambda x: plato_a_animal[x.lower()])

### Reemplazar valores
El método data.replace() ofrece varias formas de efectuar reemplazos sobre una serie de Pandas:
    1- Un valor viejo por un valor nuevo.
    2- Una lista de valores viejos por un valor nuevo.
    3- Una lista de valores viejos por una lista de valores nuevos.
    4- Un diccionario que mapee valores nuevos y viejos.

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

* Podemos hacer `replace` diferentes usando una lista de listas...

In [ ]:
data.replace([-999, -1000], [0, np.nan])

* ... O usando un `dict` 

In [ ]:
data.replace({-999: np.nan, -1000: 0})

In [ ]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                    'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})

df

In [ ]:
df.replace(0, 5)

In [ ]:
df.replace([0, 1, 2, 3], 4)

In [ ]:
df.replace({'A': 0, 'B': 5}, 100)

Se pueden usar expresiones regulares:

In [ ]:
df_re = pd.DataFrame({'A': ['bat', 'foo', 'bait'],
                      'B': ['abc', 'bar', 'xyz']})

df_re

In [ ]:
df_re.replace(to_replace=r'^ba.$', value='new', regex=True)

In [ ]:
df_re.replace(to_replace=r'^ba.+$', value='new', regex=True)

In [ ]:
df_re.replace({'A': r'^ba.$'}, 'new', regex=True)

In [ ]:
df_re.replace(regex={r'^ba.$': 'new', 'foo': 'xyz'})

### Renombrar índices de los ejes

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Buenos Aires', 'Cordoba', 'Mendoza'],
                 columns=['uno', 'dos', 'tres', 'cuatro'])
data

In [ ]:
data.index = data.index.map(str.upper)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.title.html

In [ ]:
data

In [ ]:
data.rename(index={'CORDOBA': 'SANTA FE'},
            columns={'tres': 'ocho'})

In [ ]:
data

In [ ]:
data.rename(index={'CORDOBA': 'SANTA FE'}, inplace=True)
data

### Discretizar y binarizar variables
El proceso de transformar una variable numérica en categórica se llama discretización. 

In [ ]:
ages = [26, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32, 25, 60]

* El método `cut` devuelve el intervalo semi-cerrado al que pertenece cada entrada

In [ ]:
# Defino los valores de corte
bins = [18, 25, 35, 60, 100]

# Obtengo una lista de intervalos
cats = pd.cut(ages, bins)
cats

In [ ]:
type(cats)

* El atributo `codes` representa el indice en la lista 'cats'  del intervalo al que pertenece cada entrada

In [ ]:
cats.codes

In [ ]:
pd.value_counts(cats)

In [ ]:
pd.value_counts(cats.codes)

In [ ]:
# Podemos modificar la inclusión del valor de corte en los intervalos
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
# Podemos asignar etiquetas a las categorías
group_names = ['Joven', 'Joven Adulto', 'Adulto', 'Senior']
cats.categories = group_names
cats

In [ ]:
cats.value_counts()

* También es posible asignar nombres (etiquetas) a los intervalos generados. Puede hacerse a partir del parámetro `labels=`

In [ ]:
pd.cut(ages, [18, 25, 35, 60, 100], labels=['Joven', 'Joven Adulto', 'Adulto', 'Senior'])

* Qué sucede con el órden de las etiquetas?

In [ ]:
serie_ages = pd.cut(ages, bins, labels=group_names)
serie_ages.value_counts()

### Cuantiles en lugar de intervalos preestablecidos

In [ ]:
# Divido en cuantiles, en este caso 10
data = np.random.randn(1000)
qcats = pd.qcut(data, 5) 
qcats

In [ ]:
qcats.value_counts()

### Detectar y filtrar outliers
No existe un criterio que sea válido en todos los casos para identificar los outliers. El criterio de mayor que el tercer cuartil más 1.5 veces el rango intercuartil o menor que el primer cuartil menos 1.5 veces el rango intercuartil surge de la distribución normal. En esa distribución el 99.7% de la población se encuentra en el rango definido por la media (poblacional) más menos 3 veces el desvío estándar (poblacional)


In [ ]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.sample(5)

In [ ]:
data.describe()

In [ ]:
col = data[3]
col[np.abs(col) > 3]
#en este caso la media poblacional es 0 y el desvío es 1 entonces el criterio mencionado anteriormente 
# marcaría como outliers a los valores mayores que 3 o menores que -3
#col[np.abs(col) > 3 * np.std(col)]

In [ ]:
# Listamos aquellos que no son outliers
data[~(np.abs(data) > 3).any(axis=1)].head()

In [ ]:
len(data[~(np.abs(data) > 3).any(axis=1)])

In [ ]:
# Listamos las filas que tienen elementos que están en los extremos de la distribución
data[(np.abs(data) > 3).any(1)]

In [ ]:
# Acota el rango de la muestra
# Convierte los valores extremos a esos puntos de referencia
 
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

### PARTE II: Variables categóricas y Dummies
Pandas cuenta con el método pd.get_dummies() que recibe una Serie o una lista de Series y realiza el one hot encoding.

Recordemos que una variable con k categorías se puede representar con k-1 variables.

Por eso un parámetro clave de pd.get_dummies es drop_first = True que genera k-1 categorías en lugar de k.

In [ ]:
df = pd.DataFrame({'cat_producto': ['b', 'b', 'a', 'c', 'a', 'b'],
                'cod_venta': np.arange(100, 112, 2)})
df

In [ ]:
pd.get_dummies(df['cat_producto'])

In [ ]:
# Agregamos un prefijo para identificar la categoría
pd.get_dummies(df['cat_producto'], prefix='cat_producto')
                        

In [ ]:
dummies = pd.get_dummies(df['cat_producto'], prefix='cat_producto',
                         drop_first=True)
dummies

In [ ]:
# Concatenamos la columna cod_venta
df_with_dummy = df[['cod_venta']].join(dummies)
df_with_dummy

## Manipulación de strings

### String object methods

* `split()` toma un string, lo divide en función de un delimitador (`sep`) y devuelve una lista

In [ ]:
val = 'a,b,  guido, asjd, kle'
val.split(',')

* `strip()` toma un string y devuelve un string sin los espacios iniciales y finales.

In [ ]:
# Ejemplos:

texto = "   Este es el primer ejemplo....wow!!!   ";
print(texto.strip())

texto1 = "0000000Este es el segundo ejemplo....wow!!!0000000";
print(texto1.strip( '0' ))

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

* `find()` devuelve el índice más bajo dentro de un string en el cual un substring es encontrado. Devuelve -1 si no la encuentra

In [ ]:
val.find(':')

In [ ]:
val.find('b')

* `index()` es similar, pero devuelve un `ValueError` cuando no encuentra el substring buscado

In [ ]:
val.index(',')

In [ ]:
# Genera un error si no encuentra el substring

try:
    val.index(':')
except:
    print("Error, substring no encontrado!")

* `count()` cuenta la ocurrencia de un substring determinado en un string mayor.

In [ ]:
val.count(',')

* `replace()` reemplaza un substring por otro.

In [ ]:
val.replace(',', ';')

### Funciones vectorizadas para strings en Pandas

In [ ]:
import re
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)

In [ ]:
data

In [ ]:
data.isnull()

In [ ]:
pattern = r'\w+'

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)[0][-1]

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

### Ejemplo: Dataset movies

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movies.csv', header=None,
                        names=mnames, encoding="latin9", sep=';')
movies[:10]

In [ ]:
lista = [sublista.split('|') for sublista in movies.genres]
lista[:10]

In [ ]:
# Aplanamos la lista anterior
genres = sorted(set([item for s in lista for item in s]))
genres

In [ ]:
# Creamos un DataFrame vacío para asociar los géneros correspondientes a cada película
dummies = pd.DataFrame(np.zeros((len(movies), len(genres)), dtype=int), columns=genres)
dummies.head()

In [ ]:
# Codifica las categorías como dummies. Escribe un 1 donde corresponde
for i, gen in enumerate(movies.genres):
    dummies.loc[i, gen.split('|')] = 1

In [ ]:
dummies.head()

In [ ]:
movies_final = movies.join(dummies.add_prefix('Genre_'))
movies_final.head()